<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_ask_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-8.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-8.validation.csv",header=None)
valid.head()

,0,1
0,0,scared to death
1,2,heel pain
2,5,increased heart rate
3,6,stage 1 diastolic dysfunction
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-8.test.csv",header=None)
test.head()

,0,1
0,1,Abdominal rash
1,5,increase my heart rate
2,5,tachycardia
3,8,arthritis
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_ask.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'muscle',
 'a',
 'was',
 'diclofenac',
 'with',
 'were',
 'severe',
 'for',
 'myalgia',
 'lower',
 'my',
 'fatigue',
 'loss',
 'weakness',
 'sodium',
 'cramp',
 'limb',
 'on',
 'by',
 'stomach',
 '-',
 'depression',
 'back',
 'is',
 'at',
 'cramps',
 'patients',
 'that',
 'as',
 'joint',
 'upper',
 'gas',
 'memory',
 'or',
 'leg',
 'drug',
 'mg',
 'legs',
 'pains',
 'knee',
 'headache',
 'from',
 'shoulder',
 'unable',
 'group',
 'not',
 'arthralgia',
 'gastrointestinal',
 'neck',
 'study',
 'foot',
 'treatment',
 'excessive',
 'all',
 'arthritis',
 'aches',
 'this',
 'hip',
 'after',
 'voltaren',
 'insomnia',
 'numbness',
 'disease',
 'arthrotec',
 'release',
 'p',
 'nausea',
 'increased',
 'both',
 '/',
 'bleeding',
 'asthenia',
 'walk',
 'abdominal',
 'swelling',
 'be',
 'an',
 'heart',
 'like',
 'tired',
 'feeling',
 'skin',
 'dizzines

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.792647,3.397962,0.456852,28:37


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.426883,3.337075,0.461825,28:22


In [25]:
learn.save_encoder('pubmed_ask_fold8_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_ask_fold8_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.041767,3.736209,0.347719,01:12


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.876636,3.073824,0.471023,01:17


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.327642,2.600571,0.537608,03:16


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.927378,2.195226,0.598027,03:09
1,1.686485,2.064573,0.628853,03:29


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.518019,1.826842,0.670777,03:24
1,1.304574,1.725335,0.681874,03:12


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.199337,1.611043,0.692972,03:28
1,1.044897,1.531832,0.711467,03:15


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.993573,1.473663,0.725031,03:21
1,0.905567,1.384297,0.747226,03:33


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.848265,1.382123,0.741060,03:19
1,0.750232,1.315451,0.759556,03:20


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.660527,1.279337,0.764488,03:10
1,0.694273,1.238804,0.773120,03:16
2,0.580396,1.194737,0.790382,03:21
3,0.506445,1.180726,0.786683,03:19


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.479620,1.165596,0.787916,03:18
1,0.549712,1.156211,0.790382,03:13
2,0.454509,1.131837,0.795314,03:23
3,0.393537,1.100782,0.796547,03:17


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.386579,1.141480,0.796547,03:11
1,0.396734,1.187671,0.801480,03:16
2,0.371621,1.137319,0.802713,03:19
3,0.311613,1.159460,0.806412,03:16


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.294896,1.133405,0.810111,03:26
1,0.359980,1.178244,0.807645,03:27
2,0.299985,1.189804,0.807645,03:10
3,0.266452,1.120640,0.816276,03:16


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.267845,1.173342,0.811344,03:12
1,0.295779,1.186801,0.800247,03:15
2,0.266194,1.144637,0.810111,03:27
3,0.237783,1.130272,0.817509,03:29


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.229019,1.169161,0.806412,03:19
1,0.277244,1.210645,0.815043,03:21
2,0.244095,1.232033,0.812577,03:19
3,0.219720,1.196922,0.823674,03:22


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.213150,1.237890,0.815043,03:27
1,0.275595,1.245178,0.816276,03:28
2,0.219506,1.214066,0.817509,03:15
3,0.184941,1.235495,0.822441,03:14


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.206056,1.185138,0.805179,03:31
1,0.205152,1.207410,0.811344,03:12


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.226010,1.269972,0.806412,03:13
1,0.183113,1.266676,0.811344,03:24


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.188199,1.263678,0.815043,03:13


In [44]:
# save the best model

learn.save_encoder('pubmed_ask_mixed_fold8')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[384, 18, 424, 8, 8, 8, 12, 20, 162, 27, 491, 491, 31, 31, 34, 65, 655, 487, 37, 37, 37, 37, 42, 926, 49, 775, 49, 49, 775, 615, 100, 53, 53, 53, 53, 53, 60, 60, 60, 70, 74, 76, 76, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 80, 80, 80, 80, 85, 87, 91, 91, 91, 94, 94, 94, 77, 47, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 30, 94, 234, 1021, 427, 100, 100, 100, 100, 100, 100, 998, 105, 108, 108, 108, 842, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 112, 117, 118, 585, 125, 125, 125, 125, 125, 125, 125, 125, 125, 20, 289, 801, 127, 135, 94, 805, 94, 387, 89, 137, 139, 142, 847, 151, 151, 152, 157, 159, 67, 165, 166, 166, 770, 172, 2, 172, 172, 173, 173, 181, 182, 182, 182, 182, 182, 182, 698, 183, 183, 183, 183, 183, 183, 183, 183, 184, 184, 185, 243, 187, 187, 187, 651, 189, 190, 190, 190, 190, 190, 190, 190, 190, 111, 193, 170, 834, 206

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

674
0.7782909930715936
